#### About the data source

Because this is a proprietary data source, it will be available to RadiumOne only.  Please contact  me for access.

In [ ]:
#import model_dict
import pandas as pd
import csv as csv
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import sklearn as sk
import time
import sys
%pylab inline

sys.path.append('/path/to/model_dict.py/')

import model_dict
rcParams['figure.figsize'] = 20, 5
current_date = time.strftime("%Y%m%d")

### Instructions!

- place the following in a fresh directory:
    
    1. this iPython notebook
    2. Nicole's grid search module: model_dict.py
    3. data: hit_retarget_site_ua_conversion.csv
    
    
- in the same directory, make a folder called "models".  Your models will be placed in this folder automatically
- run the below code and explore the data!

In [ ]:
# read in the data (should be in same directory as this ipython notebook)
df_raw = pd.read_csv('/path/to/data.csv', sep = '\t', header = 0)
df_raw.columns = [...]

In [ ]:
# see breakdown of successes and failures.
# currently configured to print breakdown by stratifying 'ua_family'.
# recommended to check breakdown by stratifying on other features as well
_,_ = model_dict.success_breakdown(df_raw, by_column='ua_family', nan_breakdown=True)

### Now use model_dict.py module to do grid search on thousands of random forests

In [ ]:
params = {'max_depth':[5,6,7,8,9,10, None],
          'max_features': range(2, len(features)),
          'min_samples_leaf':[1,2,4,6],
          'criterion': ['gini', 'entropy'],
          'class_weight':['auto', 'subsample', None]}

rf = RandomForestClassifier(n_estimators=100)

### To make models, execute the following code block
**(make sure to comment out after so you don't repeat the grid search)**

Should take about 40 minutes to run for a 25,000 row dataset

In [ ]:
models = model_dict.ModelDict(prefix='%s_modelsearch_example' % current_date, params=params, initialization='make', estimator = rf)
models.make_models(x_train, y_train)

### Once models are made, comment out the above cell. 
From now on, you can use the below cell to retrieve them

In [ ]:
#models = model_dict.ModelDict(prefix='%s_amex_rf_withsite' % current_date, params=params, initialization='retrieve', estimator = rf)

In [ ]:
models.display_models()

### Check model performance


In [ ]:
mlpl = model_dict.MlPlot(models.model_dict, x_test = x_test, y_test = y_test, evaluation_metrics = ['f1', 'recall', 'precision', 'f1_weighted'])
mlpl.display_performance()

### Predict performance (for use on your holdout set of data)

Here I'll predict performance on the test set.  You'll make a holdout set before doing the test/train split above, and then score that here.

In [ ]:
%pylab inline

best_model = models.__getmodel__('{0}_modelsearch_example_f1'.format(current_date))
prediction = [p[1] for p in best_model.predict_proba(x_test[features])]

#visualize the results in a histogram
histogram = hist(prediction)